# 単体テスト

## 統計データ取得

In [ ]:
import urllib.parse
import urllib.request
import json

def get_estat_response(params):
  # appId
  params['appId'] = '724e5b90772a3e9289f41a253e4e7e32438f4fff'
  
  # url生成
  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(params)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

In [ ]:
params = {
    'statsDataId': '0000010101',
    'cdCat01': 'A1101',
    }

res = get_estat_response(params)
print(res)

{'GET_STATS_DATA': {'RESULT': {'STATUS': 0, 'ERROR_MSG': '正常に終了しました。', 'DATE': '2022-12-25T08:11:58.147+09:00'}, 'PARAMETER': {'LANG': 'J', 'STATS_DATA_ID': '0000010101', 'NARROWING_COND': {'CODE_CAT01_SELECT': 'A1101'}, 'DATA_FORMAT': 'J', 'START_POSITION': 1, 'METAGET_FLG': 'Y'}, 'STATISTICAL_DATA': {'RESULT_INF': {'TOTAL_NUMBER': 2160, 'FROM_NUMBER': 1, 'TO_NUMBER': 2160}, 'TABLE_INF': {'@id': '0000010101', 'STAT_NAME': {'@code': '00200502', '$': '社会・人口統計体系'}, 'GOV_ORG': {'@code': '00200', '$': '総務省'}, 'STATISTICS_NAME': '都道府県データ 基礎データ', 'TITLE': {'@no': '0000010101', '$': 'Ａ\u3000人口・世帯'}, 'CYCLE': '年度次', 'SURVEY_DATE': 0, 'OPEN_DATE': '2022-03-04', 'SMALL_AREA': 0, 'MAIN_CATEGORY': {'@code': '99', '$': 'その他'}, 'SUB_CATEGORY': {'@code': '99', '$': 'その他'}, 'OVERALL_TOTAL_NUMBER': 500976, 'UPDATED_DATE': '2022-03-04', 'STATISTICS_NAME_SPEC': {'TABULATION_CATEGORY': '都道府県データ', 'TABULATION_SUB_CATEGORY1': '基礎データ'}, 'TITLE_SPEC': {'TABLE_NAME': 'Ａ\u3000人口・世帯'}}, 'CLASS_INF': {'CLASS_OBJ'

### categories

In [ ]:
import pandas as pd

def df_categories(res: dict) :
  CLASS_OBJ = res['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']
  categories = list(filter(lambda x: x['@id'] == 'cat01', CLASS_OBJ))[0]['CLASS']
  
  # DataFrameに変換
  df = pd.DataFrame(categories) if type(categories) is list else pd.DataFrame([categories])

  # 列の整理
  columns = {'@code':'categoryCode','@name':'categoryName'}
  df = df[list(columns.keys())].rename(columns = columns)
  
  # categoryNameから不要な情報（categoryCode）を削除
  df['categoryName'] = df.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)

  return df

In [ ]:
df_categories(res)

,categoryCode,categoryName
0,A1101,総人口


### areas

In [ ]:
def df_areas(res: dict) :
  CLASS_OBJ = res['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']
  areas = list(filter(lambda x: x['@id'] == 'area', CLASS_OBJ))[0]['CLASS']

  # DataFrameに変換
  df = pd.DataFrame(areas) if type(areas) is list else pd.DataFrame([areas])

  # 列の整理
  columns = {'@code':'areaCode','@name':'areaName'}
  df = df[list(columns.keys())].rename(columns = columns)

  return df

In [ ]:
df_areas(res)

### times

In [ ]:
def df_times(res: dict) -> list:
  CLASS_OBJ = res['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']
  times = list(filter(lambda x: x['@id'] == 'time', CLASS_OBJ))[0]['CLASS']

  # DataFrameに変換
  df = pd.DataFrame(times) if type(times) is list else pd.DataFrame([times])

  # 列の整理
  columns = {'@code':'timeCode','@name':'timeName'}
  df = df[list(columns.keys())].rename(columns = columns)

  return df

In [ ]:
df_times(res)

,timeCode,timeName
0,1975100000,1975年度
1,1976100000,1976年度
2,1977100000,1977年度
3,1978100000,1978年度
4,1979100000,1979年度
5,1980100000,1980年度
6,1981100000,1981年度
7,1982100000,1982年度
8,1983100000,1983年度
9,1984100000,1984年度


### formated Data

In [ ]:
import pandas as pd

def df_data(res):
  VALUE = res['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']
  
  # DataFrameに変換
  df = pd.DataFrame(VALUE)

  # 欠損値の処理
  df = df.dropna().replace('-', '0').replace('X', '0')
  
  # 列の整理
  columns = {'@cat01':'categoryCode','@area':'areaCode','@time':'timeCode','$':'value','@unit':'unit'}
  df = df[list(columns.keys())].rename(columns = columns)
  
  # merge
  df = pd.merge(df, df_categories(res), on='categoryCode', how='left')
  df = pd.merge(df, df_areas(res), on='areaCode', how='left')
  df = pd.merge(df, df_times(res), on='timeCode', how='left')
  
  # 型変換
  if isinstance(float(df.iloc[1,3]), int):
    df = df.astype({'value': int})
  else:
    df = df.astype({'value': float})

  return df

In [ ]:
params = {
    'statsDataId': '0000010101',
    'cdCat01': 'A1231',
    }
res = get_estat_response(params)

# print(df_data(res).dtypes)
df_data(res).head()

,categoryCode,areaCode,timeCode,value,unit,categoryName,areaName,timeName
0,A1231,00000,2010100000,45.0,歳,年齢中位数,全国,2010年度
1,A1231,00000,2015100000,46.7,歳,年齢中位数,全国,2015年度
2,A1231,01000,2010100000,47.7,歳,年齢中位数,北海道,2010年度
3,A1231,01000,2015100000,49.7,歳,年齢中位数,北海道,2015年度
4,A1231,02000,2010100000,49.1,歳,年齢中位数,青森県,2010年度


## メタ情報

In [ ]:
import urllib.parse
import urllib.request
import json

def get_estat_meta(statsDataId):
  params ={
      'statsDataId': statsDataId,
      'appId': '724e5b90772a3e9289f41a253e4e7e32438f4fff',
  }

  url = 'http://api.e-stat.go.jp/rest/3.0/app/json/getMetaInfo?'
  url += urllib.parse.urlencode(params)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

In [ ]:
meta = get_estat_meta('0000010101')
print(meta)

### カテゴリ情報

In [ ]:
def catelgoryCodes(statsDataId):
  # メタ情報の取得
  meta= get_estat_meta(statsDataId)

  # CLASS_OBJ
  CLASS_OBJ = meta['GET_META_INFO']['METADATA_INF']['CLASS_INF']['CLASS_OBJ']
  cat01 = next((d for d in CLASS_OBJ if d['@id'] == 'cat01'), None)['CLASS']
  
  return [d.get('@code') for d in cat01]

In [ ]:
print(catelgoryCodes('0000010101'))

## BigQueryへの登録

### ライブラリ

pandas-gbqではなくgoogle-cloud-bigqueryを使う

https://cloud.google.com/bigquery/docs/pandas-gbq-migration?hl=ja

In [ ]:
!pip install --upgrade 'google-cloud-bigquery[bqstorage,pandas]'
!pip install pyarrow

### GCP認証

In [ ]:
from google.colab import auth
auth.authenticate_user()

### プロジェクト設定

In [ ]:
project_id='primal-buttress-342908'
dataset_id = 'estat_origin'

### GBQに保存する関数

In [ ]:
from google.cloud import bigquery

def estat_to_gbq(params):
  # estatレスポンスの取得
  res = get_estat_response(params)
  
  # Dataframe作成  
  df = df_data(res)
  
  # table設定
  table_id = params['statsDataId'] + '_' + params['cdCat01']
  
  # スキーマ
  job_config = bigquery.LoadJobConfig(write_disposition='WRITE_TRUNCATE')

  # bigqueryへ保存
  client = bigquery.Client(project=project_id)
  job = client.load_table_from_dataframe(df, f'{dataset_id}.{table_id}', job_config=job_config).result()

  # 確認メッセージ  
  table = client.get_table(f'{dataset_id}.{table_id}') 
  print("{} 件のデータを「{}」に保存しました。".format(table.num_rows, table_id))
  
  return

In [ ]:
params = {
    'statsDataId': '0000020201',
    'cdCat01': 'A1101',
    }

estat_to_gbq(params)

15313 件のデータを「0000020201_A1101」に保存しました。


# 一括処理

In [ ]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

def save_gbq(data):
  for statsDataId in data:
    for c in catelgoryCodes(statsDataId):
      params = {
          'statsDataId': statsDataId,
          'cdCat01': c,
      }

      client = bigquery.Client(project=project_id)
      table_id = params['statsDataId'] + '_' + params['cdCat01'] 
      
      try:
        client.get_table(f'{dataset_id}.{table_id}')  
        print("Table {} already exists.".format(table_id))
      except NotFound:
        estat_to_gbq(params)


## 社会・人口統計体系（都道府県）

https://www.e-stat.go.jp/stat-search/database?page=1&layout=datalist&toukei=00200502&tstat=000001111375&cycle=8&tclass1=000001111377&tclass2val=0

In [ ]:
# 統計表のリスト
data = ['0000010101', '0000010102', '0000010103', '0000010104', '0000010105', '0000010106', '0000010107', 
        '0000010108', '0000010109', '0000010110', '0000010111', '0000010112', '0000010113']

## メタ情報をGBQに保存

## データをGBQに保存

In [ ]:
save_gbq(data)

## 社会・人口統計体系（市区町村）

https://www.e-stat.go.jp/stat-search/database?page=1&layout=datalist&toukei=00200502&tstat=000001111376&cycle=8&tclass1=000001111380&collect_area=200&result_page=1&tclass2val=0

In [ ]:
# 統計表のリスト
data = ['0000020201', '0000020202', '0000020203', '0000020204', '0000020205', '0000020206', 
        '0000020207', '0000020208', '0000020209', '0000020210', '0000020211']
save_gbq(data)

Table 0000020201_A1101 already exists.
7668 件のデータを「0000020201_A110101」に保存しました。
7668 件のデータを「0000020201_A110102」に保存しました。
7668 件のデータを「0000020201_A1102」に保存しました。
7668 件のデータを「0000020201_A110201」に保存しました。
7668 件のデータを「0000020201_A110202」に保存しました。
3834 件のデータを「0000020201_A1231」に保存しました。
15313 件のデータを「0000020201_A1301」に保存しました。
7668 件のデータを「0000020201_A130101」に保存しました。
7668 件のデータを「0000020201_A130102」に保存しました。
15313 件のデータを「0000020201_A1302」に保存しました。
7668 件のデータを「0000020201_A130201」に保存しました。
7668 件のデータを「0000020201_A130202」に保存しました。
15313 件のデータを「0000020201_A1303」に保存しました。
7668 件のデータを「0000020201_A130301」に保存しました。
7668 件のデータを「0000020201_A130302」に保存しました。
7668 件のデータを「0000020201_A1401」に保存しました。
7668 件のデータを「0000020201_A140101」に保存しました。
7668 件のデータを「0000020201_A140102」に保存しました。
7668 件のデータを「0000020201_A1402」に保存しました。
7668 件のデータを「0000020201_A140201」に保存しました。
7668 件のデータを「0000020201_A140202」に保存しました。
7668 件のデータを「0000020201_A1403」に保存しました。
7668 件のデータを「0000020201_A140301」に保存しました。
7668 件のデータを「0000020201_A140302」に保存しました。
7668 件のデータを「00